## 第六章 神经网络学习相关技巧

### 6.1参数的更新

In [6]:
class SGD:#随机梯度法更新权重参数,缺点是低效且梯度方向并没有指向最小的方向。
    def _init_(self,lr=0.01):
        self.lr=lr
        
    def update(self,params,grads):
        for key in params.keys():
            params[key]-=self.lr*grads[key]
        

In [8]:
class Momentum:#类似于物理中的动量，给予权重减速的效果。
    def _init_(self,lr=0.01,momentum=0.9):
        self.lr=lr
        self.momentum=momentum
        self.v=None
        
    def update(self,params,grads):
        if self.v is None:
            self.v={}
            for key,val in params.items():
                self.v[key]=np.zerps_like(val)
        for key in params.keys():
            self.v[key]=self.momentum*self.v[key]-self.lr*grads[key]
            params[key]+=self.v[key]
    
        

In [10]:
class AdaGrad:#将不同的学习率分别递减，使变动大的学习率逐渐减小。
    def _init_(self,lr=0.01):
        self.lr=lr
        self.h=None
        
    def update(self,params,grads):
        if self.h is None:
            self.h={}
            for key,val in params.items():
                self.h[key]=np.zerps_like(val)
    for key in params.keys():
        self.h[key]+=grads[key]*grads[key]
        params[key]-=self.lr*grads[key]/(np.sqrt(self.h[key])+1e-7)#防止除以0。

NameError: name 'params' is not defined

## 6.2 权重的初始值设定

### 标准差为0.01的随机高斯分布；Xavier初始值，激活函数一般使用sigmoid函数或tanh函数；
### He初始值，一般使用Relu函数。

### Batch Norm正则化参数，使数据分布的均值为0，方差为1的正规化。使数据分布拥有适当的广度。
### 学习更快的进行。之后再对正规化后的数据进行平移变换，参数可以通过学习调整为适当值。

## 6.3 抑制过拟合

### 权值衰减：为损失函数加上权重的平方范数，意在学习过程中对去权值较大的参数惩罚抑制过拟合。
### Dropout：学习过程中随机删除神经元，测试时，对于神经元处理，乘以训练时的删除比例再输出。

In [11]:
class Dropout:
    def _init_(self,dropout_ratio=0.5):
        self.dropout_ratio=dropout_ratio
        self.mask=None
    def forward(self,x,train_flg=True):
        if train_flg:
            self.mask=np.random.rand(*x.shape)>self.dropout_ratio
            return x*self.mask
        else:
            return x*(1-self.dropout_ratio)
    def backward(self,dout):
        return dout*self.mask